# Running the experiments of the paper

## 1. Run the assimilator double gyre experiments on double gyre files

In [ ]:
# Download the data, if necessary

import wget
from pathlib import Path

files_to_download = ["parts_double_gyre_ref_eps_0.25_A_0.1175_as_2.nc", "parts_double_gyre_ref_eps_0.25_A_0.105_as_2.nc",
"parts_double_gyre_ref_eps_0.25_A_0.11_as_2.nc",
"parts_double_gyre_ref_eps_0.25_A_0.125_as_2.nc",
"parts_double_gyre_ref_eps_0.25_A_0.1_as_2.nc",
"parts_double_gyre_ref_eps_0.35_A_0.1_as_2.nc",
"parts_double_gyre_ref_eps_0.3_A_0.1_as_2.nc",
"parts_double_gyre_ref_eps_0.4_A_0.1_as_2.nc",
"parts_double_gyre_ref_eps_0.5_A_0.1_as_2.nc",
"parts_double_gyre_ref_eps_0_A_0.1_as_2.nc",
"parts_double_gyre_ref_eps_1_A_0.1_as_2.nc",
"parts_double_gyre_standard_as_3.nc"]

Path("data/").mkdir(exist_ok=True)
for file_to_download in files_to_download:
  if not Path(f"data/{file_to_download}").exists():
    print(f"Downloading {file_to_download}...")
    wget.download(f"https://zenodo.org/record/4426130/files/{file_to_download}", f"data/{file_to_download}")

print("All files required for experiment have been downloaded")

In [ ]:
from src.run_assimilator import run_assimilator

# Change the following parameters to reproduce the experimental results
# Changing A and epsilon will require to have downloaded the corresponding files.
# If you are trying values of A and epsilon that do not have a corresponding dispersion file, you will have to generate it with the double gyre code

mu = 2
A = 0.1175
epsilon = 0.25
sigma_rel = 0.01

if __name__ == "__main__":
    run_assimilator(
        particles_dataset_path="data/parts_double_gyre_standard_as_3.nc",
        observations_type="from_simulation",
        observations_source_path=f"data/parts_double_gyre_ref_eps_{epsilon}_A_{A}_as_2.nc",
        simulation_name=f"mu_{mu}_A_{A}_eps_{epsilon}_sigma_rel_{sigma_rel}",
        assimilation_domain_coords=(195, 20, 225, 40),
        assimilation_grid_size=(60, 40),
        size_ensemble=10,
        initial_ensemble_spread_percent=0.05,
        t_start=0,
        t_end=2000,
        observations_error_percent=sigma_rel,
        observation_locations=[(12, 4), (55, 27)],
        initial_mass_multiplicator=mu,
    )

## 2. Run a self-assimilation experiment with observation sampled a dispersion model assimilated into another dispersion model

In [ ]:
# First, prepare the data

import xarray as xr
import numpy as np
from datetime import datetime
from dask.diagnostics import ProgressBar

ds_in = xr.open_dataset("data/advector_output_coastal_2012.nc", chunks="auto")
years_release = ds_in.release_date.dt.year
days_release = ds_in.release_date.dt.dayofyear
arg_release = (years_release - 1993) + days_release / 366

def increasing_sine(x):
    return x + (1 / (2 * np.pi)) * np.sin(2 * np.pi * x + np.pi)

ds_in["weight"] = 1 + increasing_sine(arg_release)
with ProgressBar():
    ds_in.to_netcdf("data/advector_output_coastal_2012_adapted.nc")

In [ ]:
from src.run_assimilator import run_assimilator

if __name__ == "__main__":
    run_assimilator(
        particles_dataset_path="data/advector_output_rivers_2012.nc",
        observations_type="from_simulation",
        observations_source_path="data/advector_output_coastal_2012_adapted.nc",
        assimilation_domain_coords=(195 - 360, 23, 235 - 360, 45),
        assimilation_grid_size=(int(40 / 0.5), int(22 / 0.5)),
        size_ensemble=10,
        initial_ensemble_spread_percent=0.05,
        observations_error_percent=0.01,
        observation_locations=[(12, 4), (55, 27)],
        t_start=0,
        t_end=360,
        initial_mass_multiplicator=1,
        radius_observation=np.inf,
        simulation_name="self_assimilation_gpgp_inf"
    )


In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy as cart

ds_ref = xr.open_dataset("data/data_self_assimilation_gpgp/densities_ref_self_assimilation_gpgp.nc")
ds_assimilated = xr.open_dataset("data/data_self_assimilation_gpgp_inf/densities_ensemble_self_assimilation_gpgp_inf.nc")
ds_not_assimilated = xr.open_dataset("data/data_self_assimilation_gpgp_inf/densities_ensemble_self_assimilation_gpgp_inf_init.nc")

assimilated_differences_ensembles = ds_assimilated.density - ds_not_assimilated.density
assimilated_differences_end = assimilated_differences_ensembles.mean(dim="ensemble").isel(time=360)
assimilated_differences_end['lon'] = 195 + (assimilated_differences_end.lon * 0.5)
assimilated_differences_end['lat'] = 23 + (assimilated_differences_end.lat * 0.5)

assimilated_forecast = ds_assimilated.density.mean(dim="ensemble").isel(time=360)
assimilated_forecast['lon'] = 195 + (assimilated_forecast.lon * 0.5)
assimilated_forecast['lat'] = 23 + (assimilated_forecast.lat * 0.5)

fig, axes = plt.subplots(subplot_kw={'projection': ccrs.PlateCarree()}, figsize=(16,8))
axes.coastlines()
axes.add_feature(cart.feature.LAND, zorder=100, edgecolor="k")
axes.add_feature(cart.feature.OCEAN, zorder=-100, edgecolor="k")

p1 = assimilated_differences_end.transpose().plot(ax=axes, transform=ccrs.PlateCarree(), robust=True)
p1.axes.set_extent([185, 250, 15, 50])